In [11]:
# Link
# https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf

In [2]:
from datetime import datetime

import pandas as pd
import folium

In [3]:
df = pd.read_csv('test.csv')
df['datetime'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S UTC')
df['date'] = df['datetime'].dt.date
df['hour'] = df['datetime'].dt.round('H').dt.hour
#df = df.loc[df['date'].astype(str) == '2011-12-13']

df = df[['hour', 'pickup_latitude', 'pickup_longitude', 'passenger_count']]
df.columns = ['Hour', 'Latitude', 'Longitude', 'Active_Users']

df.head()

,Hour,Latitude,Longitude,Active_Users
0,13,40.763805,-73.973320,1
1,13,40.719383,-73.986862,1
2,12,40.751260,-73.982524,1
3,21,40.767807,-73.981160,1
4,21,40.789775,-73.966046,1


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 4 columns):
Hour            9914 non-null int64
Latitude        9914 non-null float64
Longitude       9914 non-null float64
Active_Users    9914 non-null int64
dtypes: float64(2), int64(2)
memory usage: 309.9 KB


In [79]:
def generateBaseMap(default_location=[40.763805, -73.973320],
                    default_zoom_start=11):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [80]:
base_map = generateBaseMap()
base_map

In [86]:
from folium.plugins import HeatMap
base_map = generateBaseMap()
HeatMap(data=df[['Latitude', 'Longitude', 'Active_Users']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist(),
       radius=14, max_zoom=13).add_to(base_map)

In [87]:
base_map

In [88]:
base_map.add_child(folium.ClickForMarker(popup='Potential Location'))

# Heatmap with time

In [125]:
df_hour_list = []
for hour in df.Hour.sort_values().unique():
    df_hour_list.append(df.loc[df.Hour == hour, ['Latitude', 'Longitude', 'Active_Users']]. \
                        groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

In [126]:
from folium.plugins import HeatMapWithTime
map_ = generateBaseMap(default_zoom_start=11.5)
HeatMapWithTime(df_hour_list, radius=35, gradient={0.2: 'blue', 0.3: 'lime', 0.4: 'orange', 0.5: 'red'},
               min_opacity=0.5, max_opacity=1, use_local_extrema=True).add_to(map_)
map_